In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Reference Links - Evolution of Deep Neural Networks

* **McCulloch-Pitts Neuron — Mankind’s First Mathematical Model Of A Biological Neuron -** https://towardsdatascience.com/mcculloch-pitts-model-5fdf65ac5dd1
* **Perceptron Learning Algorithm: A Graphical Explanation Of Why It Works -** https://towardsdatascience.com/perceptron-learning-algorithm-d5db0deab975

* **How neural networks build up their understanding of images -** https://distill.pub/2017/feature-visualization/

* **TensorFlow Extended (TFX) is an end-to-end platform for deploying production ML pipelines -** https://www.tensorflow.org/tfx

# <center> Linear Regression using TensorFlow

In this session, we will have a look at creating a linear regression model using tensorflow 2.0. Note that we already know the basics of linear regression and understand the implementation through sklearn. We will try to figure out how to do it using tensorflow tools that we have learnt.   


<i>Note that to keep this exercise simple and focused on tensorflow and its relevant functions, we will make a very simple model with very basic preprocessing.</i> 

In [ ]:
import tensorflow as tf
print(tf.__version__)

Now Let us import out data and get it ready for modelling.

In [ ]:
cars_data = pd.read_csv('/kaggle/input/dl-intro-to-nn/usedcars.csv')
cars_data

In [ ]:
cars_data.isna().sum()

In [ ]:
#just some basic preprocessing
cars_data.drop_duplicates(inplace=True)

In [ ]:
#creating dummy variables for the categorical features
cars_data = pd.get_dummies(cars_data)
cars_data = cars_data.astype('float32') # we will need to convert the dataset to float in order to be able to convert it into tensors later.
cars_data

In [ ]:
#exploring column names
cars_data.columns

In [ ]:
#getting the features and labels and finally splitting the test and train data.


from sklearn.model_selection import train_test_split
X = cars_data[['year','mileage', 'model_SE', 'model_SEL', 'model_SES',
       'color_Black', 'color_Blue', 'color_Gold', 'color_Gray', 'color_Green',
       'color_Red', 'color_Silver', 'color_White', 'color_Yellow',
       'transmission_AUTO', 'transmission_MANUAL']]
Y = cars_data['price']


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1)

In [ ]:
#let us scale the data as features are on different scales which might be a problem while modelling
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
# MinMaxScalar has been used here. You can go ahead and use the other scalars available and chcek the effect on the results.
#fitting the transform on test and train separately
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

In [ ]:

# let us now convert the data elements into tensors as we need tensors to be fed into different tensorflow based operations
#X-train and X_test were converted to numpy arrays while transformations while the other two need to be transformed into numpy arrays.
X_train=tf.convert_to_tensor(X_train)
y_train=tf.convert_to_tensor(y_train.values)
X_test=tf.convert_to_tensor(X_test)
y_test=tf.convert_to_tensor(y_test.values)

In [ ]:
X_train.shape

#### Let us try modelling now. We will use a few concepts covered in the practice exercise shared with the course material.

In [ ]:
input_dim = X_train.shape[1]
output_dim = 1
learning_rate = 0.01

# Let us initialize the weights and bias variables. 
weights = tf.Variable(tf.zeros(shape=(input_dim, output_dim), dtype= tf.float32))
bias = tf.Variable(tf.ones(shape=(output_dim,), dtype= tf.float32))

In [ ]:
weights.shape

In [ ]:
bias.shape

In [ ]:
def predict(features):
    return tf.matmul(features, weights) + bias # note that the matmul is matrix multiplication and is needed for calculating predictions

def compute_loss(y_true, predictions):
    return tf.reduce_mean(tf.square(y_true - predictions)) # mean square error

# Let us now define a function to train the model. We will call the other functions in function definition.
def train(x, y):
    with tf.GradientTape() as tape:
        predictions = predict(x)
        loss = compute_loss(y, predictions)
        dloss_dw, dloss_db = tape.gradient(loss, [weights, bias]) #note that we can pass lists as well here.
    weights.assign_sub(learning_rate * dloss_dw)
    bias.assign_sub(learning_rate * dloss_db)
    return loss

#### Let us now, call the train function with 50 epochs

In [ ]:
for epoch in range(50):
    loss = train(X_train, y_train)
    print('Epoch %d: Loss = %.4f' % (epoch, float(loss)))

In [ ]:
print('Final Weights after 50 epochs:')
print(weights)

In [ ]:
print('Final Bias after 50 epochs:')
print(bias)

#### Let us now test our model on the test data and predict on the test data.

In [ ]:
test_predictions = tf.matmul(X_test, weights) + bias
print(compute_loss(y_test, test_predictions))

We learnt creating a very simple linear regression model on cars data and predicted prices. 

## <center> ML vs DL

Deep Learning is considered as a subset of machine learning as it achieves the same goal, making machines learn from the data to gather meaningful insights. However, a common question that arises is - Why move to neural networks when we had the traditional machine learning algorithms? So let us have a look at the differences now.

We will see the differences between the two techniques based on the following parameters-

* **1. Feature Engineering**

Machine Learning - requires extensive feature engineering for the model to perform better.
Deep Learning - learns to extract the relevant features from the dataset 

* **2. Performance with data**

Deep Learning - performs poorly with lesser data but outshines the traditional machine learning algorithms with huge amounts of data. Since we are generating huge amounts of data every day, deep learning algorithms definitely perform better.

* **3. Computation Requirements**

Traditional machine learning algorithms generally have a low computational requirement. However, since there are not that much of matrix multiplications required in the implementation unlike neural networks based learning, therefore, these algorithms are computationally extensive.
    
Now, with the huge amounts of data being generated and extensive computation abilities being present, organisations are stressing on deep learning research work and most of the AI-based applications are finding their foundations in Neural Networks.

Happy Learning!